conda: pytorch_playground

In [1]:
import torch, torchvision
from torch import nn
import torch.nn.functional as F
# from torch import optim
# from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import pandas as pd

# torch.set_printoptions(linewidth=120)

# Import model

In [2]:
import models
model = models.CNN()
model.load_state_dict(torch.load("model_folder/model_2_best_1.pt"))

<All keys matched successfully>

# Import data

In [3]:
T = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
mnist_data_train = torchvision.datasets.MNIST('mnist_data',
                                              transform=T,
                                              download=True, 
                                              train=True)
mnist_data_valid = torchvision.datasets.MNIST('mnist_data',
                                             transform=T,
                                             download=True, 
                                             train=False)
emnist_data_train = torchvision.datasets.EMNIST('emnist_data',
                                               split = "balanced",
                                               transform=T,
                                               download=True,
                                               train=True)
emnist_data_valid = torchvision.datasets.EMNIST('emnist_data',
                                               split = "balanced",
                                               transform=T,
                                               download=True,
                                               train=False)

In [4]:
# target = list(range(10))
# target

# Batch

In [5]:
# bs = 2000
# mnist_dataloader_train = torch.utils.data.DataLoader(mnist_data_train, batch_size=bs)
# mnist_dataloader_valid = torch.utils.data.DataLoader(mnist_data_valid, batch_size=bs)
# emnist_dataloader_train = torch.utils.data.DataLoader(emnist_data_train, batch_size=bs)
# emnist_dataloader_valid = torch.utils.data.DataLoader(emnist_data_valid, batch_size=bs)

# m_batch = next(iter(mnist_dataloader_train))
# m_images, m_labels = m_batch

# e_batch = next(iter(emnist_dataloader_train))
# e_images, e_labels = e_batch

# MC_dropout

In [6]:
# batch = m_batch
# images, labels = batch

In [7]:
# images.shape

In [8]:
# model.eval()
# with torch.no_grad():
#     preds_eval = model(images)

# model.train()
# preds_eval_s = F.softmax(preds_eval, dim=1)

In [9]:
# n = 1000
# mc_list = []
# with torch.no_grad():
#     for i in range(n):
#         preds = model(images)
#         mc_list.append(preds.unsqueeze(dim=1))

In [10]:
# mc = torch.cat((mc_list), dim=1)
# mc_s = F.softmax(mc, dim=2)
# mc.shape

In [11]:
# img = 0
# mc_s[img].sum()

In [12]:
# mc.mean(dim=1).argmax(dim=1)

In [13]:
# plt.imshow(images[img].squeeze(), cmap = "gray")
# print("label:", labels[img])

In [14]:
# sns.stripplot(target, preds_eval[img], jitter=0)\
#     .set_title("Standard - Pre-softmax")

In [15]:
# sns.stripplot(target, preds_eval_s[img], jitter=0)\
#     .set_title("Standard - Pre-softmax")

In [16]:
# sns.stripplot(target*n, mc[img].reshape(mc[img].numel()), jitter=0.3)\
#     .set_title("MC_dropout - Pre-softmax")

In [17]:
# fig_mc_s = sns.stripplot(target*n, mc_s[img].reshape(mc_s[img].numel()), jitter=0.3)\
#     .set_title("MC_dropout - Softmax")

In [18]:
# fig_mc_b = sns.boxplot(target*n, mc_s[img].reshape(mc_s[img].numel()))
# fig_mc_b.set_title("MC_dropout - Softmax")

In [19]:
# fig_mc_b

# FINAL TRY?

## SET: n

In [20]:
I = 3

In [21]:
n = 100

In [22]:
sample = mnist_data_train[0]
image, label = sample

In [23]:
model.eval()
with torch.no_grad():
    pred_eval = model(image.unsqueeze(dim=0))

model.train()
pred_eval_s = F.softmax(pred_eval, dim=1)

In [24]:
mc_list = []
with torch.no_grad():
    for i in range(n):
        preds = model(image.unsqueeze(dim=0))
        mc_list.append(preds.unsqueeze(dim=2))

In [25]:
mc = torch.cat((mc_list), dim=2)
mc_s = F.softmax(mc, dim=1)
# mc_s[img].sum() # =n
mc.shape

torch.Size([1, 10, 100])

### GET MC VALUES

In [26]:
# np.percentile(mc_s[0][4], [0, 25, 50, 75, 100])

In [27]:
mc_s.shape

torch.Size([1, 10, 100])

In [28]:
image_values = []
for _image in mc_s:
    target_values = []
    for _target in _image:
        target_values.append(_target.mean())
        target_values.append(_target.std())
    image_values.append(target_values)
mc_tensor = torch.tensor(image_values)
mc_tensor.shape

torch.Size([1, 20])

In [29]:
# model.eval()
# with torch.no_grad():
#     preds_eval = model(images)

# model.train()
# preds_eval_s = F.softmax(preds_eval, dim=1)

In [30]:
# mc_labels = pred_eval_s.argmax(dim=1).eq(label).type_as(pred_eval_s)

In [31]:
mc_features = torch.cat((mc_tensor, pred_eval_s), dim=1)
mc_features.shape

torch.Size([1, 30])

In [32]:
X = mc_features

### Now add letters to the training data

In [33]:
# batch_e = e_batch
# images_e, labels_e = batch_e

# model.eval()
# with torch.no_grad():
#     preds_eval_e = model(images_e)

# model.train()
# preds_eval_s_e = F.softmax(preds_eval_e, dim=1)

# mc_list_e = []
# with torch.no_grad():
#     for i in range(n):
#         preds_e = model(images_e)
#         mc_list_e.append(preds_e.unsqueeze(dim=2))
        
# mc_e = torch.cat((mc_list_e), dim=2)
# mc_s_e = F.softmax(mc_e, dim=1)
# # mc_s[img].sum() # =n
# mc_e.shape

In [34]:
# image_values_e = []
# for _image_e in mc_s_e:
#     target_values_e = []
#     for _target_e in _image_e:
#         target_values_e.append(_target_e.mean())
#         target_values_e.append(_target_e.std())
#     image_values_e.append(target_values_e)
# mc_tensor_e = torch.tensor(image_values_e)
# mc_tensor_e.shape

In [35]:
# mc_labels_e = preds_eval_s.argmax(dim=1).eq(labels).type_as(preds_eval_s)
# mc_labels_e.shape

In [36]:
# # mc_labels_e = [0] * preds_eval_s.shape[0]
# mc_labels_e = torch.zeros(2000)

In [37]:
# mc_features_e = torch.cat((mc_tensor_e, preds_eval_s), dim=1)
# mc_features_e.shape

In [38]:
# mc_features_e.eq(mc_features)

In [39]:
# mc_features_e[0]

In [40]:
# mc_features

### PLOT

In [41]:
# targets = []
# for i in range(10):
#     for _ in range(n):
#         targets.append(i)

In [42]:
# img = 2

In [43]:
# fig_mc_b = sns.boxplot(targets, mc_s[img].reshape(mc_s[img].numel()))
# fig_mc_b.set_title("MC_dropout - Softmax")

In [44]:
# mc_s_e.shape

In [45]:
# fig_mc_b_e = sns.boxplot(targets, mc_s_e[img].reshape(mc_s_e[img].numel()))
# fig_mc_b_e.set_title("MC_dropout - Softmax - e")

# Combine MNIST and EMNIST data

In [46]:
# X = torch.cat((mc_features, mc_features_e))
# y = torch.cat((mc_labels, mc_labels_e))

In [47]:
# X.shape

In [48]:
# y.shape

In [49]:
# # y = np.array(y, dtype=int)
# one = 0
# zero = 0
# for i in y:
#     if i == 1:
#         one += 1
#     if i == 0:
#         zero += 1
# one, zero

### Only extract 2 highest probabilities with its stdev

In [50]:
# index = reversed(X[0][20:30].argsort())[:2]

eval probs  -  mc prob mean  -  mc prob std

In [51]:
# torch.cat((X[0][20:30][index], X[0][index*2], X[0][index*2+1]))

Loop

In [52]:
# new_X_list = []
# for i in range(len(X)):
#     index = reversed(X[i][20:30].argsort())[:2]
#     keep_values = torch.cat((X[i][20:30][index], X[i][index*2], X[i][index*2+1]))
#     new_X_list.append(keep_values.unsqueeze(dim=0))
# new_X = torch.cat(new_X_list, dim=0)

In [53]:
# new_X.shape

SAME BUT NOW WITH 5

In [54]:
new_X_2_list = []
for i in range(len(X)):
    index_2 = reversed(X[i][20:30].argsort())[:5]
    keep_values_2 = torch.cat((X[i][20:30][index_2], X[i][index_2*2], X[i][index_2*2+1]))
    new_X_2_list.append(keep_values_2.unsqueeze(dim=0))
new_X_2 = torch.cat(new_X_2_list, dim=0)

NOW ALL

In [55]:
# new_X_3_list = []
# for i in range(len(X)):
#     index_3 = reversed(X[i][20:30].argsort())
#     keep_values_3 = torch.cat((X[i][20:30][index_3], X[i][index_3*2], X[i][index_3*2+1]))
#     new_X_3_list.append(keep_values_3.unsqueeze(dim=0))
# new_X_3 = torch.cat(new_X_3_list, dim=0)

### Only stdev

In [56]:
# newer_X_list = []
# for i in range(len(X)):
#     stdevs = X[i][[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]]
#     newer_X_list.append(stdevs.unsqueeze(dim=0))
# newer_X = torch.cat(newer_X_list, dim=0)

In [57]:
# newer_X.shape

## Only stdev + odered

In [58]:
# newerer_X_list = []
# for i in range(len(X)):
#     stdevs = sorted(X[i][[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]])
#     newerer_X_list.append(torch.tensor(stdevs).unsqueeze(dim=0))
# newerer_X = torch.cat(newerer_X_list, dim=0)

In [59]:
# sorted(X[i][[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]])

In [60]:
# newerer_X.shape

# NEW MODEL

In [61]:
# !pip install sklearn

In [62]:
# import sklearn as sk 
# sk.__version__

In [63]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import balanced_accuracy_score
# from sklearn import metrics

# from sklearn import model_selection
# from sklearn.model_selection import cross_val_score

# from sklearn.linear_model import LogisticRegression
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC

# from datetime import datetime
import pickle

In [64]:
# train_X, test_X, train_y, test_y = train_test_split(newer_X, y, test_size = 0.2)

## Compare models

In [65]:
# def compare_models(X, Y, kfold_cv=10): 
#     '''
#     Comparing 6 models: LR (linear regression), LDA (logistic discriminant 
#     analysis), RF (random forest), DT (decision tree), KNN (k-nearest neighbours),
#     NB (Naive Bayes classifier) using k-fold crossvalidation. 
#     A plot will be shown to demonstrate the distribution of the accuracy values.
    
#     Input: X_train and y_train dataframes, preset: 10 fold CV.
    
#     Output: Dataframe with the 6 models and their properties.
#     '''
#     # prepare models and append the various models
#     models = []
#     models.append(('LR',  LogisticRegression()))
#     models.append(('LDA', LinearDiscriminantAnalysis()))
#     models.append(('RF',  RandomForestClassifier()))
#     models.append(('DT',  DecisionTreeClassifier()))
#     models.append(('KNN', KNeighborsClassifier()))
#     models.append(('NB',  GaussianNB()))
#     models.append(('SVM', SVC()))
#     # evaluate each model in turn
#     results = []
#     names = []
#     scoring = 'accuracy' #we use the accuracy as the estimator of the quality of the models
#     # train the models using cross validation
#     for name, model in models:
#         # prepare the cross validation
#         kfold = model_selection.KFold(n_splits=kfold_cv)
#         # get the cross validation results (where the magic happens)
#         cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
#         # append these results to the results list
#         results.append(cv_results)
#         # append the name to the name list
#         names.append(name)
#         # put it in a nice format and extract the meand and std
#         model_acc = "{}: \t{:.4f} ({:.4f})".format(name, cv_results.mean(), cv_results.std()) 
#         # print the accuracy values of the models
#         print(model_acc)
#     # boxplot algorithm comparison, make a boxplot of the results
#     fig = plt.figure()
#     fig.suptitle('Algorithm Comparison (accuracy)')
#     ax = fig.add_subplot(111)
#     plt.boxplot(results)
#     ax.set_xticklabels(names)
#     plt.show()
#     # make dataframe with the names of he model and the actual model (to be saved)
#     models_df = pd.DataFrame(models, columns = ["model", "model_info"])
#     return models_df

In [66]:
# compare_models(X, y)

In [67]:
# compare_models(new_X, y)

In [68]:
# uncertainty_models = compare_models(new_X_2, y)

In [69]:
# compare_models(new_X_3, y)

In [70]:
# compare_models(newer_X, y)

In [71]:
# compare_models(newerer_X, y)

In [72]:
# rf = RandomForestClassifier()

In [73]:
# rf.fit(new_X_2, y)

In [74]:
# plt.bar(range(len(rf.feature_importances_)), rf.feature_importances_)

In [75]:
# pickle.dump(rf, open("model_folder/rf_uncertainty_model.sav", "wb"))

In [76]:
loaded_model = pickle.load(open("model_folder/rf_uncertainty_model.sav", "rb"))

In [79]:
loaded_model.predict(new_X_2)

array([1.], dtype=float32)

In [80]:
pred_eval_s.argmax(dim=1)

tensor([5])

In [ ]:
# new_X_2[:1000]